# Settings

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil
import sys
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()
import math
import random
import torch

import warnings
warnings.filterwarnings('ignore')

In [2]:
class dummy_class:
    def __init__(self):
        pass
    
args = dummy_class()

args.seed = 42

args.summarization_input = '/train.tsv'
args.summarization_output = '/test.tsv'

args.summarization_pretrain = 'gogamza/kobart-summarization'
# args.summarization_pretrain = 'gogamza/kobart-base-v2'
args.version = 'backup8'
args.summarization_model = f'kobart/{args.version}'
args.summarization_model_b = 'kobart/backup7'
args.isBefore = True

# args.sentence_embedding_pretrain = 'BM-K/KoSimCSE-bert-multitask'
args.sentence_embedding_pretrain = 'BM-K/KoSimCSE-roberta-multitask'

args.check_percent = 0.99
args.max_len = 1024
args.ignore_index = -100

args.num_beams=5
args.max_length=512
args.bos_token_id = 0
args.eos_token_id=1
args.pad_token_id = 3
args.decoder_start_token_id = 0
args.repetition_penalty = 5.0
args.length_penalty = 1.0

args.scale = 'small' # small / middle / original
args.newSample = False

def add_padding_data(inputs):
    if len(inputs) < args.max_len:
        pad = np.array([args.pad_token_id] *(args.max_len - len(inputs)))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:args.max_len]

    return inputs

def filtering_text(text):
    try:
        
        length = len(text)
        index = set(range(length))

        pairs1 = list()
        stack = list()
        for i in range(length):
            if text[i] == '(':
                stack.append(i)
            elif text[i] == ')':
                if stack:
                    pairs1.append((stack.pop(), i))
        
        for i in stack:
            pairs1.append((i, i))
        
        pairs2 = list()
        stack = list()
        for i in range(length):
            if text[i] == '[':
                stack.append(i)
            elif text[i] == ']':
                if stack:
                    pairs2.append((stack.pop(), i))
                    
        for i in stack:
            pairs2.append((i, i))
    
    except:
        print(text)
                
    for start, end in pairs1 + pairs2:
        index -= set(range(start, end+1))

    index = list(index)
    index.sort()
    text = ''.join([text[idx] for idx in index])
            
    return text

os.environ["PYTHONHASHSEED"] = str(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True

In [3]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

In [4]:
path = '.'
kobart_data_path = path + '/kobart/data'
data_path = path + '/data'
patent_path = data_path + '/patent'
patent_training_path = patent_path + '/Training/training_특허전체'
patent_validation_path = patent_path + '/Validation/validation_특허전체'
patent_name = '/특허전체_{}-Y-{}.json'
patent_training_index = list(zip([9] * 33, range(33))) + list(zip([10] * 14, range(14)))
patent_validation_index = list(zip([10] * 5, range(14,20)))
patent_training_name = '/patent_training.tsv'
patent_validation_name = '/patent_validation.tsv'
file_name = '/patent.tsv'
save_name = '/patents_10.tsv'

class_name = '/class.tsv'

# columns = ['순서', '출원번호', '발명의명칭', '출원인', '공개번호', '원출원번호', '발명자', '관련 출원번호', '요약', '청구항']
# pd.DataFrame(columns = columns).to_csv(data_path + file_name, sep = '\t', index = False)

# columns = ['제목', '문서번호', '등록번호', '등록날짜', '요약(Input)', '요약(Output)']
# pd.DataFrame(columns = columns).to_csv(patent_training_path + patent_training_name, sep = '\t', index = False)
# pd.DataFrame(columns = columns).to_csv(patent_validation_path + patent_validation_name, sep = '\t', index = False)

# columns = ['order', 'abstract', 'number', 'class']
# pd.DataFrame(columns = columns).to_csv(data_path + class_name, sep = '\t', index = False)

# Summarization Input/Output

In [15]:
df_training = pd.read_csv(kobart_data_path + args.summarization_input, sep = '\t')
df_validation = pd.read_csv(kobart_data_path + args.summarization_output, sep = '\t')
df_training.shape, df_validation.shape

((65710, 2), (13369, 2))

In [10]:
# df_training['output'] = df_training['output'].apply(filtering_text)
# df_validation['output'] = df_validation['output'].apply(filtering_text)

In [16]:
df_training.dropna(inplace = True)
df_validation.dropna(inplace = True)
df_training.shape, df_validation.shape

((65710, 2), (13369, 2))

In [17]:
df_training['input_length'] = df_training['input'].apply(len)
df_validation['input_length'] = df_validation['input'].apply(len)

df_training['output_length'] = df_training['output'].apply(len)
df_validation['output_length'] = df_validation['output'].apply(len)

In [18]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained(args.summarization_pretrain)
df_training['input_tokens'] = df_training['input'].apply(lambda x: len(tokenizer.tokenize(x)))
df_validation['input_tokens'] = df_validation['input'].apply(lambda x: len(tokenizer.tokenize(x)))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [19]:
df_training['output_tokens'] = df_training['output'].apply(lambda x: len(tokenizer.tokenize(x)))
df_validation['output_tokens'] = df_validation['output'].apply(lambda x: len(tokenizer.tokenize(x)))

In [11]:
df_training = df_training[(df_training['input_length'] > df_training['output_length'])]
df_validation = df_validation[(df_validation['input_length'] > df_validation['output_length'])]
df_training.shape, df_validation.shape

((123265, 6), (13302, 6))

In [48]:
df_training['input_tokens'].describe()

count    65710.000000
mean       170.400487
std         75.469496
min         20.000000
25%        119.000000
50%        158.000000
75%        207.000000
max       1578.000000
Name: input_tokens, dtype: float64

In [49]:
df_training['input_tokens'].quantile(0.99)

419.0

In [15]:
df_validation['input_tokens'].describe()

count    13369.000000
mean       203.804473
std        116.523143
min         18.000000
25%        129.000000
50%        178.000000
75%        248.000000
max       1617.000000
Name: input_tokens, dtype: float64

In [16]:
df_validation['input_tokens'].quantile(0.99)

628.3199999999997

In [17]:
df_training['output_tokens'].describe()

count    124058.000000
mean         53.874462
std          36.392755
min           1.000000
25%          29.000000
50%          46.000000
75%          69.000000
max         767.000000
Name: output_tokens, dtype: float64

In [18]:
df_training['output_tokens'].quantile(0.99)

181.0

In [19]:
df_validation['output_tokens'].describe()

count    13369.000000
mean        61.222829
std         38.729763
min          1.000000
25%         35.000000
50%         54.000000
75%         78.000000
max        574.000000
Name: output_tokens, dtype: float64

In [20]:
df_validation['output_tokens'].quantile(0.99)

193.0

In [21]:
df_training.shape, df_validation.shape

((124058, 6), (13369, 6))

In [33]:
df_training[df_training['output_tokens'] == 1]

,input,output,input_length,output_length,input_tokens,output_tokens


In [34]:
df_validation[df_validation['output_tokens'] == 1]

,input,output,input_length,output_length,input_tokens,output_tokens
871,"본 발명은 컷아웃스위치 또는 피뢰기 고정용 경완금의 제조방법에 관한 것으로, FRP...",.,1319,1,640,1
3295,본 발명은 물건 지지팔의 폭 조절이 가능한 핸드 캐리어에 관한 것이다. 본 발명은 ...,.,1140,1,590,1
6738,본 발명은 대형의 화분 이동용 핸드 캐리어에 관한 것이다. 본 발명은 상단에 손잡이...,.,1691,1,879,1
7470,기계구조물용 고정 핀 빼기 및 박음장치 내부에 사각의 구멍이 형성되며 실린더의 ...,.,1128,1,564,1
11720,본 발명은 엘이디모듈의 변형을 방지한 엘이디투광등에 관한 것이다. 이러한 본 발명은...,.,869,1,472,1


In [22]:
df_training['ratio_length'] = df_training['output_length'] / df_training['input_length']
df_training['ratio_tokens'] = df_training['output_tokens'] / df_training['input_tokens']

In [23]:
((df_training['ratio_tokens'] < 0.2) | (df_training['ratio_tokens'] > 0.5)).agg(['mean', 'sum'])

mean        0.470328
sum     58348.000000
Name: ratio_tokens, dtype: float64

In [24]:
df_training.loc[1015]['input'], df_training.loc[1015]['output']

('본 발명의 과제는 감도, 해상성, 노광 마진이 우수할 뿐 아니라, MEEF 성능도 만족시키는 감방사선성 수지 조성물 및 이 감방사선성 수지 조성물에 바람직하게 이용되는 저분자 화합물을 제공하는 것이다. 본 발명은  특정 구조를 가지는 산해리성기 함유 질소 화합물,  산해리성기로 보호된 알칼리 불용성 또는 알칼리 난용성의 수지이며, 상기 산해리성기가 해리되었을 때에 알칼리 가용성이 되는 수지, 및  감방사선성 산 발생제를 포함하는 것을 특징으로 하는 감방사선성 수지 조성물을 제공한다.',
 '본 발명은 IC 등의 반도체 제조 공정, 액정, 서멀 헤드 등의 회로 기판의 제조, 그 밖의 포토리소그래피 공정에 사용되는 감방사선성 수지 조성물에 관한 것이다.  보다 구체적으로는, KrF 엑시머 레이저·ArF 엑시머 레이저 등의 파장 250 nm 이하의 원자외선이나 전자선을 노광 광원으로 하는 포토리소그래피 공정에 바람직하게 사용할 수 있는 화학증폭형의 감방사선성 수지 조성물에 관한 것이다.  ')

In [25]:
df_training.loc[1066]['input'], df_training.loc[1066]['output']

('본 발명은 아이씨카드에 관한 것으로, 투명 재질의 상층 코팅 매체부, 상기 상층 코팅 매체부 안쪽에 구비되는 상층 절연 매체부, 상기 상층 절연 매체부와 접합되도록 구비되는 하층 절연 매체부, 상기 상층 절연 매체부와 하층 절연 매체부의 접합부에 매입되는 IC칩을 포함하여 구성된 아이씨카드에 있어서, 상기 상층 코팅 매체부 안쪽의 상층 절연 매체부에 매입되어 2차원 좌표 영역을 표시하는 표시 모듈과 상기 2차원 좌표 영역에 표시될 2차원 좌표 표시 값으로 부호화 가능한 크기 이내의 부호화 대상 값을 동적으로 생성하는 생성 모듈과 상기 부호화 대상 값을 상기 2차원 좌표 영역에 표시 가능한 i개의 표시 영역 좌표 값과 j개의 공백 영역 좌표 값으로 부호화한 2차원 좌표 표시 값을 생성하는 부호화 모듈 및 상기 생성된 2차원 좌표 표시 값을 상기 표시 모듈의 2차원 좌표 영역에 표시되도록 제어하는 제어 모듈을 구비한다.',
 '본 발명은 투명 재질의 상층 코팅 매체부, 상기 상층 코팅 매체부 안쪽에 구비되는 상층 절연 매체부, 상기 상층 절연 매체부와 접합되도록 구비되는 하층 절연 매체부, 상기 상층 절연 매체부와 하층 절연 매체부의 접합부에 매입되는 IC칩을 포함하여 구성된 아이씨카드에 있어서, 상기 상층 코팅 매체부 안쪽의 상층 절연 매체부에 매입되어 2차원 좌표 영역을 표시하는 표시 모듈과 상기 2차원 좌표 영역에 표시될 2차원 좌표 표시 값으로 부호화 가능한 크기 이내의 부호화 대상 값을 동적으로 생성하는 생성 모듈과 상기 부호화 대상 값을 상기 2차원 좌표 영역에 표시 가능한 i개의 표시 영역 좌표 값과 j개의 공백 영역 좌표 값으로 부호화한 2차원 좌표 표시 값을 생성하는 부호화 모듈 및 상기 생성된 2차원 좌표 표시 값을 상기 표시 모듈의 2차원 좌표 영역에 표시되도록 제어하는 제어 모듈을 구비하는 아이씨카드에 관한 것이다.')

In [26]:
df_training[['input', 'output', 'ratio_length', 'ratio_tokens']].loc[1060:1068]

,input,output,ratio_length,ratio_tokens
1060,"도전성 분말, 과냉각 액체 구간을 가지는 금속 유리, 그리고 유기 비히클을 포함하는...","도전성 페이스트, 상기 도전성 페이스트를 사용하여 형성된 전극을 포함하는 전자 소자...",0.581818,0.566038
1061,본 발명은 백라이트 유닛과 이를 구비하는 표시 장치에 관한 것이다. 상기 백라이트 ...,본 발명은 표시 장치에 관한 것이다.,0.058140,0.047059
1062,본 발명은 백라이트 유닛과 이를 구비하는 표시 장치에 관한 것이다. 상기 백라이트 ...,본 발명은 표시 장치에 관한 것이다.,0.057307,0.047059
1063,"본 발명은 블라스팅 로봇 제어기의 방열 및 방진을 위한 장치에 관한 것으로, 함체와...","본 발명은 블라스팅 로봇 제어기의 방열 및 방진을 위한 장치에 관한 것으로, 보다 ...",0.455357,0.432558
1064,"간격조정장치를 구비하는 냉장고를 개시한다. 냉장고는 본체와, 상기 본체를 개폐하는 ...","본 발명은 냉장고에 관한 것으로, 더욱 상세하게는 냉장고 도어와 본체 사이의 간격을...",0.594406,0.545455
1065,휴대 단말에 구비된 적어도 하나의 센서로부터 센싱 데이터를 수집하는 센싱 데이터 수...,휴대 단말을 이용하여 사용자의 현재 행동 상태를 측정하는 기술과 관련된다.,0.127726,0.116667
1066,"본 발명은 아이씨카드에 관한 것으로, 투명 재질의 상층 코팅 매체부, 상기 상층 코...","본 발명은 투명 재질의 상층 코팅 매체부, 상기 상층 코팅 매체부 안쪽에 구비되는 ...",0.997835,1.000000
1067,"본 발명은 플라스틱 카드에 관한 것으로, 투명 재질의 상층 코팅 매체부, 상기 상층...","본 발명은 투명 재질의 상층 코팅 매체부, 상기 상층 코팅 매체부 안쪽에 구비되는 ...",0.997890,1.000000
1068,"전극 연장부를 갖는 발광 다이오드가 개시된다. 이 발광 다이오드는, 제1 도전형 질...","본 발명은 발광 다이오드에 관한 것으로, 특히 전극 연장부를 갖는 발광 다이오드에 ...",0.129268,0.114428


In [31]:
df_training = df_training[(df_training['ratio_tokens'] >= 0.2) & (df_training['ratio_tokens'] <= 0.5)]

In [33]:
df_training.shape, df_validation.shape

((65710, 8), (13369, 6))

In [345]:
df_training['filtered_input'] = df_training['input'].apply(filtering_text)
df_validation['filtered_input'] = df_validation['input'].apply(filtering_text)

In [346]:
df_training['filtered_input_length'] = df_training['filtered_input'].apply(len)
df_validation['filtered_input_length'] = df_validation['filtered_input'].apply(len)

In [347]:
df_training['filtered_input_tokens'] = df_training['filtered_input'].apply(lambda x: len(tokenizer.tokenize(x)))
df_validation['filtered_input_tokens'] = df_validation['filtered_input'].apply(lambda x: len(tokenizer.tokenize(x)))

In [348]:
df_training[['input_tokens','filtered_input_tokens']].describe()

,input_tokens,filtered_input_tokens
count,124653.000000,124653.000000
mean,183.773419,173.433965
std,97.856304,89.840450
min,16.000000,6.000000
25%,122.000000,116.000000
50%,166.000000,158.000000
75%,223.000000,210.000000
max,6275.000000,5092.000000


In [349]:
df_training['filtered_input_tokens'].quantile(0.99)

483.0

In [350]:
df_validation[['input_tokens','filtered_input_tokens']].describe()

,input_tokens,filtered_input_tokens
count,13401.000000,13401.000000
mean,213.830162,203.585329
std,129.355337,116.498018
min,24.000000,18.000000
25%,133.000000,129.000000
50%,185.000000,178.000000
75%,259.000000,248.000000
max,2186.000000,1617.000000


In [351]:
df_validation['filtered_input_tokens'].quantile(0.99)

628.0

In [34]:
df_training[['input', 'output']].to_csv(patent_path + '/train.tsv', sep = '\t', index = False)
df_validation[['input', 'output']].to_csv(patent_path + '/test.tsv', sep = '\t', index = False)

In [88]:
df_training[['요약(Input)', '요약(Output)']].rename({'요약(Input)':'input', '요약(Output)':'output'}, axis = 1).to_csv(patent_path + '/train.tsv', sep = '\t', index = False)
df_validation[['요약(Input)', '요약(Output)']].rename({'요약(Input)':'input', '요약(Output)':'output'}, axis = 1).to_csv(patent_path + '/test.tsv', sep = '\t', index = False)

In [352]:
df_training[['filtered_input', 'output']].rename({'filtered_input':'input'}, axis = 1).to_csv(patent_path + '/train.tsv', sep = '\t', index = False)
df_validation[['filtered_input', 'output']].rename({'filtered_input':'input'}, axis = 1).to_csv(patent_path + '/test.tsv', sep = '\t', index = False)

# Summarization

In [20]:
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers import T5TokenizerFast, T5ForConditionalGeneration

model_b = BartForConditionalGeneration.from_pretrained(args.summarization_model_b)
model = BartForConditionalGeneration.from_pretrained(args.summarization_model)
tokenizer = PreTrainedTokenizerFast.from_pretrained(args.summarization_pretrain)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [21]:
if 'input_tokens' not in df_training.columns:
    df_training['input_tokens'] = df_training['input'].apply(lambda x: len(tokenizer.tokenize(x)))
    df_validation['input_tokens'] = df_validation['input'].apply(lambda x: len(tokenizer.tokenize(x)))

df_training['input_tokens'].quantile(args.check_percent), df_validation['input_tokens'].quantile(args.check_percent)

(419.0, 628.3199999999997)

In [22]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm

In [23]:
StreamlitPatcher().jupyter()

In [24]:
df_validation.sort_values('input_tokens').tail(50)

,input,output,input_length,output_length,input_tokens,output_tokens
4504,"본 발명은, 분체혼합처리공법을 시공하기 위해 혼합처리장비의 리더 정면에 설치된 교반...",본 발명은 혼합처리장비의 교반축에 분말형 재료들을 1.5 Shot 방식으로 공급하는...,1663,299,848,142
8054,GPS 유도 파라포일 공중운송 시스템이 개시된다. 항공기로부터 투하되어 GPS 항법...,"본 발명은 GPS 유도 파라포일 공중운송 시스템에 관한 것으로 특히, GPS 신호를...",1745,90,850,40
2959,"본 발명은 바이오차 및 미생물을 이용한 상토 조성물, 토양개량제, 유기질비료 및 그...","본 발명은 바이오차 및 미생물을 이용한 상토 조성물, 토양개량제, 유기질비료 및 그...",1816,185,856,87
4058,본 발명은 물저장 화분 받침대를 구비한 적층식 식물 재배 장치 에 대한 것이다. 보...,본 발명은 물저장 화분 받침대를 구비한 적층식 식물 재배 장치 에 대한 것이다. 보...,1655,248,856,111
620,"본 발명은, 다이아몬드 파우더를 이용한 연삭공구, 그리고 이의 제조 방법에 관한 것...","본 발명은 다이아몬드 파우더를 이용한 연삭공구, 그리고 이의 제조 방법에 관한 것으...",1755,348,857,162
5015,본 발명은 활선고리를 갖고 주전력선에 설치되는 분기슬리브의 우측에 설치되는 나팔모양...,"본 발명은 배전용 주상변압기 활선고리 안전장치에 관한 것으로서, 보다 상세하게는 활...",1487,195,866,95
1933,"본 발명은 통합된 마이크로 컨트롤러 보드를 이용한 교육 시스템에 관한 것으로서, 아...","본 발명은 통합된 마이크로 컨트롤러 보드를 이용한 교육 시스템에 관한 것으로서, 보...",1829,154,870,63
13220,본 발명은 돌출 테두리부가 구비된 비구면 광학렌즈의 제조방법 및 그 제조장치에 관한...,본 발명은 돌출 테두리부가 구비된 비구면 광학렌즈의 제조방법 및 그 제조장치에 관한...,1570,246,873,123
6738,본 발명은 대형의 화분 이동용 핸드 캐리어에 관한 것이다. 본 발명은 상단에 손잡이...,.,1691,1,879,1
4974,본 개시는 치료사의 눈이나 감각에 의해 측정되어 왔던 환자의 상태를 정량적으로 진단...,"본 명세서에 개시된 내용은 하지 재활로봇의 제어장치 및 제어방법에 관한 것으로, 특...",2129,230,885,98


In [25]:
temp = df_validation.loc[6180] # 6409, 6180
# temp = df_training.loc[4126]
text = temp['input']
target = temp['output']

# text = filtering_text(text)

In [26]:
st.title("KoBART 요약 Test")
st.markdown("## 특허 요약 원문")
st.write(text)

if text:
    with st.spinner('processing..'):
        input_ids = tokenizer.encode(text)
        st.markdown("## Token 개수")
        st.write(len(input_ids))
        input_ids = add_padding_data(input_ids)
        # input_ids = [tokenizer.bos_token_id] + input_ids + [tokenizer.eos_token_id]
        input_ids = torch.tensor(input_ids).unsqueeze(0)
        attention_mask = input_ids.ne(3).float()
        
        if args.isBefore:
            summary_ids_b = model_b.generate(input_ids,  num_beams=args.num_beams,  max_length=args.max_length,  bos_token_id = args.bos_token_id, eos_token_id=args.eos_token_id, \
                                             pad_token_id = args.pad_token_id, attention_mask = attention_mask, decoder_start_token_id = args.decoder_start_token_id, \
                                             repetition_penalty = args.repetition_penalty, length_penalty = args.length_penalty)
            output_b = tokenizer.decode(summary_ids_b.squeeze().tolist(), skip_special_tokens=True)
    
        summary_ids = model.generate(input_ids,  num_beams=args.num_beams,  max_length=args.max_length,  bos_token_id = args.bos_token_id, eos_token_id=args.eos_token_id, \
                                             pad_token_id = args.pad_token_id, attention_mask = attention_mask, decoder_start_token_id = args.decoder_start_token_id, \
                                             repetition_penalty = args.repetition_penalty, length_penalty = args.length_penalty)
        output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    
    if args.isBefore:
        st.markdown("## KoBART 요약 결과(before)")
        st.write(output_b)
    
        st.markdown("## KoBART 요약 결과(after)")
    else:
        st.markdown("## KoBART 요약 결과")
    st.write(output)
    
st.markdown("## 특허 실제 요약")
st.write(target)

# KoBART 요약 Test

## 특허 요약 원문

본 발명은 가공배전선로의 케이블 탈선 방지용 거치대에 관한 것이다. 이러한 본 발명은, 완철에 착탈가능하게 설치되는 지지부와, 원호 방향을 따라 형성되는 다수 개의 관통홀을 갖추고서 지지부의 상부에 구비되되, 서로 대향되게 구비되는 한 쌍의 지탱부를 갖춘 지지체와; 원호 방향을 따라 형성되어 지탱부의 관통홀과 연통되는 다수 개의 고정홀을 구비하여 지탱부 사이에 회전가능하게 삽입되는 삽입체와, 상기 삽입체의 상단에 구비되는 제1볼트체와, 일방향으로 회전 시 제1볼트체 및 제2볼트체가 가까워지고 타방향으로 회전 시 제1볼트체 및 제2볼트체가 멀어지도록 양단이 제1볼트체 및 제2볼트체에 각각 나사산 결합되는 길이조절너트체 및, 길이조절너트체에 이동가능하게 나사산 결합되는 제2볼트체를 갖춘 길이조절기구 및; 지지체의 관통홀 및 길이조절기구의 고정홀을 관통하는 고정볼트와, 고정볼트에 나사산 결합되며, 고정볼트의 헤드부에 대향되도록 지탱부의 외측에 배치되는 고정너트를 갖춘 고정기구를 구비한 지지기구, 지지기구의 제2볼트체의 상부에 착탈가능하게 설치되는 분기슬리브커버, 분기선삽입부의 중앙부에 전후 방향으로 개구되게 형성되고 좌측이 개구되는 안착홀과, 분기선삽입부의 하부에 전후 방향으로 개구되게 형성되고 상측이 개구되어 상기 안착홀과 연통되며 상기 안착홀의 하방에 구비되는 제1삽입홈과, 우측이 개구되어 분기선삽입부의 둘레를 따라 형성되는 제2삽입홈과, 분기선삽입부의 우측부에 좌우 방향으로 개구되게 형성되며 좌측이 상기 안착홀과 연통되고 우측이 상기 제2삽입홈과 연통되는 삽입홀과, 분기선삽입부의 하부에 좌우 방향으로 개구되게 형성되고 우측이 상기 제1삽입홈과 연통되며 제1중간부재을 가압하기 위한 볼트가 이동가능하게 나사산 결합되는 제1나사홀과, 분기선삽입부의 상부에 전후 방향으로 개구되게 형성되고 상측이 개구되는 제3삽입홈 및, 분기선삽입부의 상부 우측부에 좌우로 개구되게 형성되고 좌측이 상기 제3삽입홈과 연통되며 우측이 상기 제2삽입홈과 연통되어 제2중간부재을 가압하기 위한 볼트가 이동가능하게 나사산 결합되는 제2나사홀을 갖춘 분기선삽입부와; 점퍼선삽입부의 중앙부에 전후 방향으로 개구되게 형성되고 우측이 개구되는 안착홀과, 우측이 개구되어 점퍼선삽입부의 둘레를 따라 형성되는 삽입홈 및, 점퍼선삽입부의 우측부에 좌우 방향으로 개구되게 형성되며 좌측이 상기 안착홀과 연통되고 우측이 상기 삽입홈과 연통되는 삽입홀을 구비하여, 분기선삽입부의 상부에 위치하는 점퍼선삽입부를 갖춘 분기슬리브본체와: 좌우 방향으로 개구된 걸림홀을 구비하여 분기선삽입부의 제1삽입홈에 슬라이딩가능하게 삽입되는 삽입부와, 상기 삽입부로부터 연장되어 분기선삽입부의 안착홀에 삽입되는 곡판 형상의 곡판부를 구비하며, 분기슬리브본체와 동일 재질로 이루어진 제1판재와; 좌우 방향으로 개구되어 분기선삽입부의 제1나사홀 및 상기 제1판재의 걸림홀과 연통되는 걸림홀을 구비하여 분기선삽입부의 제1삽입홈에 슬라이딩가능하게 삽입되는 삽입부와, 상기 삽입부로부터 연장되며 분기선삽입부의 안착홀에 삽입되는 곡판 형상의 곡판부를 구비하며, 제1판재의 내측에 고정배치되는 구리 재질의 제2판재를 갖춘 제1중간부재과: 좌우 방향으로 개구된 걸림홀을 구비하여 분기선삽입부의 제3삽입홈에 슬라이딩가능하게 삽입되는 삽입부와, 상기 삽입부로부터 연장되며 점퍼선삽입부의 안착홀에 삽입되는 곡판 형상의 곡판부를 구비하며, 분기슬리브본체와 동일 재질로 이루어진 제1판재와; 좌우 방향으로 개구되어 분기선삽입부의 제2나사홀 및 상기 제1판재의 걸림홀과 연통되는 걸림홀을 구비하여 분기선삽입부의 제3삽입홈에 슬라이딩가능하게 삽입되는 삽입부와, 상기 삽입부로부터 연장되며 점퍼선삽입부의 안착홀에 삽입되는 곡판 형상의 곡판부를 구비하며, 제1판재의 내측에 고정배치되는 구리 재질의 제2판재를 갖춘 제2중간부재와: 표면에 스크래치가 형성되며 분기선삽입부의 제2삽입홈 및 점퍼선삽입부의 삽입홈에 삽입되는 탄성재질의 고정밴드와; 돌기를 구비하여 고정밴드에 이동가능하게 삽입되는 다수의 제1연결체와; 고정밴드의 좌측에 구비되는 볼트체와; 고정밴드의 우측에 구비되는 제2연결체 및; 좌측이 제2연결체에 회전가능하게 설치되며 우측이 볼트체에 나사산 결합되는 너트체를 갖춘 밴딩기구 및: 제1연결체의 돌기에 착탈가능하게 나사산 결합되는 제1삽입홀과, 상기 제1삽입홀과 대향되게 구비되어 제1연결체의 돌기에 착탈가능하게 나사산 결합되는 제2삽입홀을 갖추고서, 분기선삽입부의 삽입홀 및 점퍼선삽입부의 삽입홀에 각각 삽입되어, 제1중간부재의 제1판재의 곡판부 및 제2중간부재의 제1판재의 곡판부를 각각 가압하는 탄성재질인 각각의 가압기구를 갖추고서, 분기슬리브커버의 내부에 설치되는 분기슬리브, 분기슬리브커버의 상부에 설치되어 초음파를 출력하는 초음파발생기 를 포함하는 것을 특징으로 한다.

2023-07-28 01:06:18.756 
  command:

    streamlit run /opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


## Token 개수

...

본 발명은 가공배전선로의 케이블 탈선 방지용 거치대에 관한 것으로서, 보다 상세하게는, 완철에 착탈가능하게 설치되는 지지부와, 원호 방향을 따라 형성되는 다수 개의 관통홀을 갖추고서 지지부의 상부에 구비되되, 서로 대향되게 구비되는 한 쌍의 지탱부를 갖춘 지지체와, 고정볼트에 나사산 결합되는 제2볼트체를 갖춘 길이조절기구와, 상기 안착홀과 길이조절기구의 고정홀을 관통하는 고정볼트와, 고정볼트에 나사산 결합되며, 고정볼트의 헤드부에 대향되도록 지탱부의 외측에 배치되는 고정너트를 갖춘 고정기구를 구비한 분기슬리브본체와, 상기 삽입부의 중앙부에 전후 방향으로 개구되게 형성되고 좌측이 개구되는 안착홀과, 분기선삽입부의 하부에 전후 방향으로 개구되게 형성되고 상측이 개구되어 상기 안착홀과 연통되는 제1삽입홈과, 우측이 개구되어 분기선삽입부의 둘레를 따라 형성되는 제2삽입홈과, 분기선삽입부의 좌우 방향으로 개구되게 형성되며 좌측이 상기 안착홀과 연통되고, 우측이 상기 안착홀과 연통되는 제1삽입홈과, 분기선삽입부의 상하부에 좌우 방향으로 개구되게 형성되고 좌측이 상기 안착홀과 연통된 제1삽입홈과, 분기선삽입부의 하부에 좌우 방향으로 개구되게 형성되고 우측이 상기 안착홀과 연통되는 제1삽입홈과, 분기선삽입부의 하부에 좌우 방향으로 개구되게 형성되고 우측이 상기 제1삽입홈과 연통되는 제1중간부재을 가압하기 위한 볼트가 이동가능하게 나사산 결합되는 제1나사홀과, 분기선삽입부의 상부에 전후 방향으로 개구되게 형성되고 상측이 개구되는 제3삽입홈 및, 분기선삽입부의 상부 우측부에 좌우로 개구되게 형성되고 좌측이 상기 제3삽입홈과 연통되며 우측이 상기 제3삽입홈과 연통되며 우측이 상기 제2삽입홈과 연통되는 제1가사홀과, 우측이 상기 제2삽입홈과 연통되어 점퍼선삽입부의 하부에 전후 방향으로 개구되게 형성되고 상측이 개구되어 상기 안착홀과 연통되는 제2삽입홈과, 상기 안착홀과 연통되는 제1삽입홈과,

## KoBART 요약 결과(after)

본 발명은 가공배전선로의 케이블 탈선 방지용 거치대에 관한 것으로서, 보다 상세하게는, 완철에 착탈가능하게 설치되는 지지부와, 원호 방향을 따라 형성되는 다수 개의 관통홀을 갖추고서 지지부의 상부에 구비되되, 서로 대향되게 구비되는 한 쌍의 지탱부를 갖춘 지지체와, 길이조절너트체 및 길이조절너트체에 이동가능하게 나사산 결합되는 제2볼트체를 갖춘 길이조절기구 및 이를 포함하는 고정기구에 관한 것이다.

## 특허 실제 요약

본 발명은 가공배전선로의 케이블 탈선 방지용 거치대에 관한 것이다.

In [27]:
def kobart_summarization(text):
    input_ids = tokenizer.encode(text)
    input_ids = add_padding_data(input_ids)
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    attention_mask = input_ids.ne(3).float()

    summary_ids = model.generate(input_ids,  num_beams=args.num_beams,  max_length=args.max_length,  bos_token_id = args.bos_token_id, eos_token_id=args.eos_token_id, \
                                             pad_token_id = args.pad_token_id, attention_mask = attention_mask, decoder_start_token_id = args.decoder_start_token_id, \
                                             repetition_penalty = args.repetition_penalty, length_penalty = args.length_penalty)
    output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    
    return output

In [21]:
# df_validation['summarized'] = df_validation['input'].progress_apply(kobart_summarization) #   2%|▏         | 269/13369 [09:17<7:32:32,  2.07s/it] 

In [22]:
df_validation['encoded_input'] = df_validation['input'].progress_apply(tokenizer.encode)

100%|██████████| 13369/13369 [00:06<00:00, 2053.01it/s]


In [23]:
total = df_validation['encoded_input'].progress_apply(add_padding_data)
total = torch.tensor([list(value) for value in total.values])
print(total.shape)

100%|██████████| 13369/13369 [00:00<00:00, 15173.03it/s]


torch.Size([13369, 1024])


In [24]:
from math import ceil

model = BartForConditionalGeneration.from_pretrained(args.summarization_model).to('cuda')

batch_size = 32
outputs = list()

for i in tqdm(range(ceil(total.shape[0] / batch_size))):
    batch = total[i*batch_size:(i+1)*batch_size].to('cuda')
    attention_mask = batch.ne(3).float()
    summary_ids = model.generate(batch,  num_beams=args.num_beams,  max_length=args.max_length,  bos_token_id = args.bos_token_id, eos_token_id=args.eos_token_id, \
                                         pad_token_id = args.pad_token_id, attention_mask = attention_mask, decoder_start_token_id = args.decoder_start_token_id, \
                                         repetition_penalty = args.repetition_penalty, length_penalty = args.length_penalty)
    outputs += summary_ids.detach().cpu().tolist()

  0%|          | 0/418 [00:00<?, ?it/s]

In [26]:
df_validation['summarized'] = outputs
df_validation

,input,output,input_length,output_length,input_tokens,output_tokens,encoded_input,summarized
0,이 고안은 특정인들이 이용하는 이동수단에 승차한 장애인을 위해 상체 보호 구성부를 ...,"본 발명은 장애인 전용 차량에 관한 것으로서, 보다 자세하게는 보장구를 이용하여 이...",154,143,61,59,"[14025, 14068, 17662, 16886, 16707, 21229, 155...","[0, 14025, 14068, 17662, 16886, 16707, 21229, ..."
1,"본 발명은 물 없이 샴푸가 가능한 드라이 샴푸 조성물에 관한 것으로, 더욱 구체적으...",본 발명은 물 없이도 샴푸가 가능한 드라이 샴푸 조성물에 관한 것이다.,272,39,132,19,"[14285, 14103, 16237, 14198, 14807, 1700, 1124...","[0, 14285, 14103, 16237, 14198, 14807, 1700, 1..."
2,본 발명은 점검자가 쉽게 접근할 수 없는 장소 및 원거리에 균열이 발생되어 있을 때...,본 발명은 구조물 점검자가 간편하게 접근할 수 없는 원거리에 발생된 균열의 폭을 ...,438,386,200,180,"[14285, 14103, 16237, 16832, 14401, 15993, 293...","[0, 14285, 14103, 16237, 16832, 14401, 15993, ..."
3,"본 발명에 따르는, 무안경 입체영상시청을 위해 사용자 단말과 렌티큘러 렌즈 간 정렬...",본 발명은 무안경 입체영상시청을 위해 사용자 단말과 렌티큘러 렌즈 간 정렬 오차를 ...,515,190,240,87,"[14285, 14103, 16926, 28727, 243, 14113, 11699...","[0, 14285, 14103, 16237, 14113, 11699, 9085, 1..."
4,"본 발명은 친환경성 , 인체무해성, 내후성 및 대면적 코팅이 가능하면서도 플라스틱...",본 발명은 플라스틱 소재용 유기-무기 복합코팅제 조성물 및 그의 제조방법에 관한 것...,165,133,84,62,"[14285, 14103, 16237, 18043, 11280, 15004, 140...","[0, 14285, 14103, 16237, 19224, 244, 24433, 18..."
...,...,...,...,...,...,...,...,...
13364,본 발명에 따른 타일 시공 장치는 길게 형성되고 길이가 조절되도록 일부에 신축부가 ...,본 발명은 대상 벽면에 타일을 시공하기 위한 장치와 이를 이용하여 시공하는 방법에 ...,293,130,129,56,"[14285, 14103, 16926, 15176, 14339, 12041, 228...","[0, 14285, 14103, 16237, 14339, 12041, 22895, ..."
13365,얼룩 검사 장치가 제공된다. 얼룩 검사 장치는 제1 색 성분의 파장대의 투과율이 제...,본 발명은 기판의 얼룩 검사 장치 및 검사 방법에 관한 것이다.,236,35,120,16,"[14479, 10397, 17535, 14137, 15787, 14626, 990...","[0, 14285, 14103, 16237, 14479, 10397, 17535, ..."
13366,"본 발명은 하이 스피드 영상 컬리브레이션이 가능한 LED 전광판에 관한 것으로, 그...","본 발명은 하이 스피드 영상 컬리브레이션이 가능한 LED 전광판에 관한 것으로, 보...",597,304,292,143,"[14285, 14103, 16237, 17239, 19840, 9989, 1689...","[0, 14285, 14103, 16237, 17239, 19840, 9989, 1..."
13367,"본 발명은 현재 주로 사용되고 있는 움직임보상-변환부호화 영상압축기술을, 컴퓨터영상...","본 발명은 영상의 부호화/복호화에 관한 것으로, 보다 상세하게는 변환 생략 여부를 ...",396,81,181,39,"[14285, 14103, 16237, 14475, 16471, 14442, 145...","[0, 14285, 14103, 16237, 18524, 28764, 244, 10..."


In [27]:
df_validation['summarized_abstract'] = df_validation['summarized'].apply(tokenizer.decode, skip_special_tokens=True)
df_validation

,input,output,input_length,output_length,input_tokens,output_tokens,encoded_input,summarized,summarized_abstract
0,이 고안은 특정인들이 이용하는 이동수단에 승차한 장애인을 위해 상체 보호 구성부를 ...,"본 발명은 장애인 전용 차량에 관한 것으로서, 보다 자세하게는 보장구를 이용하여 이...",154,143,61,59,"[14025, 14068, 17662, 16886, 16707, 21229, 155...","[0, 14025, 14068, 17662, 16886, 16707, 21229, ...",이 고안은 특정인들이 이용하는 이동수단에 승차한 장애인을 위해 상체 보호 구성부를 ...
1,"본 발명은 물 없이 샴푸가 가능한 드라이 샴푸 조성물에 관한 것으로, 더욱 구체적으...",본 발명은 물 없이도 샴푸가 가능한 드라이 샴푸 조성물에 관한 것이다.,272,39,132,19,"[14285, 14103, 16237, 14198, 14807, 1700, 1124...","[0, 14285, 14103, 16237, 14198, 14807, 1700, 1...","본 발명은 물 없이 샴푸가 가능한 드라이 샴푸 조성물에 관한 것으로, 더욱 구체적으..."
2,본 발명은 점검자가 쉽게 접근할 수 없는 장소 및 원거리에 균열이 발생되어 있을 때...,본 발명은 구조물 점검자가 간편하게 접근할 수 없는 원거리에 발생된 균열의 폭을 ...,438,386,200,180,"[14285, 14103, 16237, 16832, 14401, 15993, 293...","[0, 14285, 14103, 16237, 16832, 14401, 15993, ...",본 발명은 점검자가 쉽게 접근할 수 없는 장소 및 원거리에 균열이 발생되어 있을 때...
3,"본 발명에 따르는, 무안경 입체영상시청을 위해 사용자 단말과 렌티큘러 렌즈 간 정렬...",본 발명은 무안경 입체영상시청을 위해 사용자 단말과 렌티큘러 렌즈 간 정렬 오차를 ...,515,190,240,87,"[14285, 14103, 16926, 28727, 243, 14113, 11699...","[0, 14285, 14103, 16237, 14113, 11699, 9085, 1...",본 발명은 무안경 입체영상시청을 위해 사용자 단말과 렌티큘러 렌즈 간 정렬 오차를 ...
4,"본 발명은 친환경성 , 인체무해성, 내후성 및 대면적 코팅이 가능하면서도 플라스틱...",본 발명은 플라스틱 소재용 유기-무기 복합코팅제 조성물 및 그의 제조방법에 관한 것...,165,133,84,62,"[14285, 14103, 16237, 18043, 11280, 15004, 140...","[0, 14285, 14103, 16237, 19224, 244, 24433, 18...","본 발명은 유기-무기 복합 코팅제 조성물과 그의 제조방법에 관한 것으로, 보다 상세..."
...,...,...,...,...,...,...,...,...,...
13364,본 발명에 따른 타일 시공 장치는 길게 형성되고 길이가 조절되도록 일부에 신축부가 ...,본 발명은 대상 벽면에 타일을 시공하기 위한 장치와 이를 이용하여 시공하는 방법에 ...,293,130,129,56,"[14285, 14103, 16926, 15176, 14339, 12041, 228...","[0, 14285, 14103, 16237, 14339, 12041, 22895, ...","본 발명은 타일 시공 장치 및 이를 이용한 타일 시공방법에 관한 것으로서, 보다 상..."
13365,얼룩 검사 장치가 제공된다. 얼룩 검사 장치는 제1 색 성분의 파장대의 투과율이 제...,본 발명은 기판의 얼룩 검사 장치 및 검사 방법에 관한 것이다.,236,35,120,16,"[14479, 10397, 17535, 14137, 15787, 14626, 990...","[0, 14285, 14103, 16237, 14479, 10397, 17535, ...","본 발명은 얼룩 검사 장치에 관한 것으로서, 보다 상세하게는 포토 레지스트가 도포된..."
13366,"본 발명은 하이 스피드 영상 컬리브레이션이 가능한 LED 전광판에 관한 것으로, 그...","본 발명은 하이 스피드 영상 컬리브레이션이 가능한 LED 전광판에 관한 것으로, 보...",597,304,292,143,"[14285, 14103, 16237, 17239, 19840, 9989, 1689...","[0, 14285, 14103, 16237, 17239, 19840, 9989, 1...","본 발명은 하이 스피드 영상 컬리브레이션이 가능한 LED 전광판에 관한 것으로, 더..."
13367,"본 발명은 현재 주로 사용되고 있는 움직임보상-변환부호화 영상압축기술을, 컴퓨터영상...","본 발명은 영상의 부호화/복호화에 관한 것으로, 보다 상세하게는 변환 생략 여부를 ...",396,81,181,39,"[14285, 14103, 16237, 14475, 16471, 14442, 145...","[0, 14285, 14103, 16237, 18524, 28764, 244, 10...","본 발명은 움직임보상-변환부호화 영상압축기술에 관한 것으로, 더욱 상세하게는 현재 ..."


In [28]:
from rouge import Rouge

def calculate_rouge(output, target, measure, case):
    try:
        rouge = Rouge()
        scores = rouge.get_scores(output, target, avg=True)
        return scores[f'rouge-{case}'][measure]
    except:
        print(output, target)
        return np.nan

df_validation['rouge_r_l'] = df_validation[['summarized_abstract', 'output']].progress_apply(lambda x : calculate_rouge(x[0], x[1], 'r', 'l'), axis = 1)
df_validation['rouge_p_l'] = df_validation[['summarized_abstract', 'output']].progress_apply(lambda x : calculate_rouge(x[0], x[1], 'p', 'l'), axis = 1)
df_validation['rouge_f_l'] = df_validation[['summarized_abstract', 'output']].progress_apply(lambda x : calculate_rouge(x[0], x[1], 'f', 'l'), axis = 1)

  8%|▊         | 1006/13369 [00:01<00:14, 849.02it/s]

본 발명은 컷아웃스위치 또는 피뢰기 고정용 경완금의 제조방법에 관한 것으로, 더욱 상세하게는 종래의 경완금에 비하여 부식이 없이 반영구적인 수명을 유지할 수 있고 중량이 가벼워 작업하기가 용이하며 강도와 탄성이 향상된 컷아웃스위치 또는 피뢰기 고정용 경완금의 제조방법에 관한 것이다. .


 25%|██▌       | 3395/13369 [00:04<00:12, 816.43it/s]

본 발명은 물건 지지팔의 폭 조절이 가능한 핸드 캐리어에 관한 것으로서, 더욱 상세하게는 손잡이가 고정되고 좌우 대칭으로 직립되는 수직 파이프와 보강 파이프의 위쪽과 중앙에 용접으로 고정되는 보강쇠, 보강쇠에 용접으로 고정되는 최소 2개의 보강 파이프, 수직 파이프의 하단 후방에 용접으로 고정되어 바퀴 지지축을 지지하는 브라키트로 되는 프레임과 브라키트 사이에 수평으로 조립되는 바퀴 지지축 및 바퀴 지지축의 양쪽 끝에 조립되는 바퀴와 보스의 역할을 겸하는 버팀쇠가 구비된다. .


 50%|█████     | 6712/13369 [00:08<00:07, 848.46it/s]

...

 57%|█████▋    | 7591/13369 [00:09<00:07, 804.61it/s]

본 발명은 고정 핀이 박혀 있는 기계구조물을 해체하여 공장 안으로 간단히 이동시킬 수 없는 상황에 있는 특수장치에 사용되는 고정 핀 빼기 및 박음장치에 관한 것으로, 더욱 상세하게는 유압실린더의 작동으로 봉체가 기계구조물에 억지끼움이 되어 있는 고정 핀에 압력을 가함과 동시에 유압브레이커의 작동에 의하여 봉체가 고정 핀이 기계구조물로부터 빠질 때까지 타격하는 고정 핀 빼기 및 박음장치에 관한 것이다. .


 89%|████████▊ | 11835/13369 [00:14<00:01, 893.15it/s]

본 발명은 엘이디투광등에 관한 것으로서, 보다 상세하게는 나사에 의해 하우징에 조립되는 탑 커버의 돌출 턱이 투명 커버의 돌출단을 누르고 투명 커버의 돌출단은 엘이디모듈의 가장자리 전체를 눌러서 하우징의 함몰부의 바닥 면에 고정된 엘이디모듈의 가장자리가 휘어져 위쪽으로 변형되지 않고 엘이디모듈의 저면 전체가 바닥 면에 거의 고르게 접촉된 상태가 유지되도록 하여 엘이디모듈로부터 발생된 열이 하우징에 잘 전달되어 엘이디모듈의 열이 양호하게 식을 수 있도록 함에 따라 엘이디모듈의 열이 더욱 크게 향상될 수 있는 엘이디투광등에 관한 것이다. .


100%|██████████| 13369/13369 [00:16<00:00, 810.52it/s]


In [30]:
df_validation[['rouge_r_l', 'rouge_p_l', 'rouge_f_l']].agg(['mean', 'std'])

,rouge_r_l,rouge_p_l,rouge_f_l
mean,0.624578,0.600332,0.583205
std,0.262716,0.288986,0.257278


In [28]:
pd.read_csv(data_path + f'/valid_backup8.tsv', sep = '\t')[['rouge_r_l', 'rouge_p_l', 'rouge_f_l']].agg(['mean', 'std'])

,rouge_r_l,rouge_p_l,rouge_f_l
mean,0.624578,0.600332,0.583205
std,0.262716,0.288986,0.257278


In [29]:
pd.read_csv(data_path + f'/valid_backup7.tsv', sep = '\t')[['rouge_r_l', 'rouge_p_l', 'rouge_f_l']].agg(['mean', 'std'])

,rouge_r_l,rouge_p_l,rouge_f_l
mean,0.623840,0.597862,0.582203
std,0.261779,0.287908,0.256280


In [30]:
pd.read_csv(data_path + f'/valid_backup6.tsv', sep = '\t')[['rouge_r_l', 'rouge_p_l', 'rouge_f_l']].agg(['mean', 'std'])

,rouge_r_l,rouge_p_l,rouge_f_l
mean,0.608929,0.650642,0.582582
std,0.270517,0.319690,0.272051


In [31]:
pd.read_csv(data_path + f'/valid_backup4.tsv', sep = '\t')[['rouge_r_l', 'rouge_p_l', 'rouge_f_l']].agg(['mean', 'std'])

,rouge_r_l,rouge_p_l,rouge_f_l
mean,0.610929,0.652523,0.585522
std,0.273706,0.321880,0.275179


In [31]:
df_validation.to_csv(data_path + f'/valid_{args.version}.tsv', sep = '\t', index = False)

In [33]:
from rouge import Rouge

def rouge_f_r_summary(hypotheses, references):
    rouge = Rouge()

    scores = rouge.get_scores(hypotheses, references, avg=True)
    rouge_f_1 = scores['rouge-1']['f']
    rouge_r_1 = scores['rouge-1']['r']
    rouge_p_1 = scores['rouge-1']['p']
    rouge_f_2 = scores['rouge-2']['f']
    rouge_r_2 = scores['rouge-2']['r']
    rouge_p_2 = scores['rouge-2']['p']
    rouge_f_L = scores['rouge-l']['f']
    rouge_r_L = scores['rouge-l']['r']
    rouge_p_L = scores['rouge-l']['p']
    return rouge_f_1, rouge_f_2, rouge_f_L, rouge_r_1, rouge_r_2, rouge_r_L, rouge_p_1, rouge_p_2, rouge_p_L

rouge_f_1, rouge_f_2, rouge_f_L, rouge_r_1, rouge_r_2, rouge_r_L, rouge_p_1, rouge_p_2, rouge_p_L = rouge_f_r_summary(output, target)
print("--------------------------Rouge-F1_Score---------------------------")
print("Rouge-F_1:", rouge_f_1)
print("Rouge-F_2:", rouge_f_2)
print("Rouge-F_L:", rouge_f_L)
print("")
print("--------------------------Rouge-Recall---------------------------")
print("Rouge-R_1:", rouge_r_1)
print("Rouge-R_2:", rouge_r_2)
print("Rouge-R_L:", rouge_r_L)
print("--------------------------Rouge-Precision---------------------------")
print("Rouge-P_1:", rouge_p_1)
print("Rouge-P_2:", rouge_p_2)
print("Rouge-P_L:", rouge_p_L)

--------------------------Rouge-F1_Score---------------------------
Rouge-F_1: 0.3508771900892583
Rouge-F_2: 0.31578947102493077
Rouge-F_L: 0.3508771900892583

--------------------------Rouge-Recall---------------------------
Rouge-R_1: 1.0
Rouge-R_2: 1.0
Rouge-R_L: 1.0
--------------------------Rouge-Precision---------------------------
Rouge-P_1: 0.2127659574468085
Rouge-P_2: 0.1875
Rouge-P_L: 0.2127659574468085


In [ ]:
# 백업 / 실험환경 / 정체성을 까먹음 / Measure 설정